# Motivation
论文原文：[《From Louvain to Leiden: guaranteeing well-connected communities》](https://arxiv.org/abs/1810.08473)

参考学习地址：[万物皆网络，万字长文详解社区发现算法Leiden](https://zhuanlan.zhihu.com/p/712714461)

代码库：https://github.com/vtraag/leidenalg

Louvain算法可能产生任意非连通的社区。非连通指代如下所示，节点 0-6 初始可以认为 1-3 和 4-6 通过 0 进行强连接，也即节点0为社区的中心。若此时 0 移动到其他社区（其他外部社区与0的连接更紧密），原图中应分为两个社区，也即 1-3 和 4-6，但因为Louvain仅考虑单个节点的移动，此时仍认为 1-6 同属一个社区，造成此时社区内不连通。

<img style="display: block; margin: 0 auto;" src="../../../assets/images/leiden-0.png" width = "600" height = "300" alt="Leiden-0" align=center />

相对louvain的优化：
* 运行速度提升：louvain算法对社区内每个顶点都尝试和其他所有社区进行模块度计算，而<font color="red">**leiden仅针对不稳定点和它直接相连的社区**</font>进行模块度计算
* 改善不良连接：Leiden 算法能够分割集群 (即分区细化)，而不是像 Louvain 算法那样只合并它们。通过以特定方式拆分集群，Leiden 算法保证了集群之间的良好连接和子集最优特性(即不可能通过将一个或多个节点从一个集群移动到另一个集群来提高集群的质量 -->通过模块度衡量)

Leiden算法在完成初步社区划分后，会先进入细化（refinement）阶段——对当前已划分好的每个社区，在其内部再次执行局部移动算法，将其进一步拆分成更小的子社区；然后才进行网络压缩，将这些细化后的子社区分别凝聚为新的节点；最终在新的压缩网络上，重复执行局部移动来优化社区结构，从而进入下一轮迭代。

## 模块度
Leiden算法使用扩展模块度公式，以处理不同密度的社区：
$$
\begin{align}
Q=\frac{1}{2m}\sum_{ij}[A_{ij}- \textcolor{red}{\gamma} \frac{k_ik_j}{2m}]\delta(c_i,c_j)
\end{align}
$$
对比原论文中公式为$H=\frac{1}{2m}\sum_c[e_c-\gamma \frac{K_c^2}{2m}]$。其中，$\gamma > 0$是调节社区内部和社区之间连接密度的分辨率参数，可以理解成相对随机分配的难度，$\gamma$越大，提升难度越大，越倾向随机，社区更小更多（分辨率越高）：
* 当 $\gamma > 1$时，会形成更多，更小且连接紧密的社区；
* 当 $\gamma < 1$时，会形成更少，更大且连接相对不那么紧密的社区。

## 算法流程

<img style="display: block; margin: 0 auto;" src="../../../assets/images/leiden-1.png" width = "800" height = "800" alt="Leiden-1" align=center />
对比Louvain，Leiden主要增加Refine步骤对划分结果进行改善，防止社区内部出现不良连接。

1. 局部移动节点：对比Louvain中一直遍历图中所有节点，直到没有任何节点移动能增加模块度；Leiden遍历图中所有节点一次后，只会访问那些邻居节点发生变化的节点。实现使用队列（先入先出），初始化时按随机顺序将图中所有节点添加至队列，然后重复步骤（1）从队列最前端移除一个节点；2）将该节点分配至最大$\Delta Q$的不同社区，若$\Delta Q<0$则保留该节点在原社区；2）如果该节点被移动到另一个社区，将所有不属于该节点新社区且不在队列中的邻居节点添加到队列尾端，因为要访问所有变化了的邻居节点）直到队列为空。
   1. 每个节点单独作为一个社区
   2. 节点移动得到社区划分
2. 社区优化（Refine）：目标是确保社区具有良好的连通性，即使被划分为子社区，仍属于同一个社区内。初始化时，b）中每个节点都在自己社区中，优化不遵循贪心，与哪个社区合并增益越大，越容易划分至对应子社区，可能将节点与随机选择的社区合并，随机程度通过参数$\color{red}{\gamma}$控制
   1. Refine，得到细化的社区划分
   2. 基于细化划分创建聚合网络，使用非细化划分创建聚合网络的初始部分，例如 b）中红色社区在 c）中被分为了两个子社区，在聚合后变为了 d）中的两个独立节点，这两个节点同属于一个社区
3. 社区压缩
   1. 进一步，Leiden移动聚合网络中各个节点e）在上图的情况，细化不改变社区划分（i.e. 2-a）
   2. 重复直至模块度无提升


|优点|缺点|
|:---|:---------|
|||
|高效，通过队列和邻接不稳定控制，减少整体无效遍历次数|分辨率参数选择敏感|
|可扩展，通过分辨率参数适应不同类型的网络结构|局部最优解，算法可能陷入局部最优解，特别在处理overlap社区的复杂网络结构|
|鲁棒性，对噪声和局部连接的影响相对小|对初始分区敏感|